<a href="https://colab.research.google.com/github/iamemc/PD_01/blob/main/PD_202021_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining / Prospeção de Dados

## Diogo Soares and Sara C. Madeira, 2020/21

# Project 1 - Pattern Mining

## Logistics 
**_Read Carefully_**

**Students should work in teams of 2 or 3 people**. 

**TASK 3 - Spring vs Summer Purchases** must be done only by groups of 3 people.

Individual projects might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `March, 28th (23:59)`.** 

Students should **upload a `.zip` file** containing all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the zip file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202021_P1.ipynb`as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs** (File > Download as > HTML).

**Decisions should be justified and results should be critically discussed.** 

_Project solutions containing only code and outputs without discussions will achieve a maximum grade 10 out of 20._

## Dataset and Tools



In this project you will analyse data from an online Store collected over 4 months (April - July 2014). The folder `data` contains three files that you should use to obtain the dataset to be used in pattern mining. 

The file `store-buys.dat` comprises the buy events of the users over the items. It contains **318.444 sessions**. Each record/line in the file has the following fields (with this order): 

* **Session ID** - the id of the session. In one session there are one or many buying events. Could be represented as an integer number.
* **Timestamp** - the time when the buy occurred. Format of YYYY-MM-DDThh:mm:ss.SSSZ
* **Item ID** – the unique identifier of item that has been bought. Could be represented as an integer number. 
* **Price** – the price of the item. Could be represented as an integer number.
* **Quantity** – the quantity in this buying.  Could be represented as an integer number.

The file `store-clicks.dat` comprises the clicks of the users over the items. It contains **5.613.499 sessions**.  Each record/line in the file has the following fields (with this order):

* **Session ID** – the id of the session. In one session there are one or many clicks. Could be represented as an integer number.
* **Timestamp** – the time when the click occurred. Format of YYYY-MM-DDThh:mm:ss.SSSZ
* **Item ID** – the unique identifier of the item that has been clicked. Could be represented as an integer number.
* **Context** – the context of the click. The value "S" indicates a special offer, "0" indicates  a missing value, a number between 1 to 12 indicates a real category identifier,
any other number indicates a brand. E.g. if an item has been clicked in the context of a promotion or special offer then the value will be "S", if the context was a brand i.e BOSCH,
then the value will be an 8-10 digits number. If the item has been clicked under regular category, i.e. sport, then the value will be a number between 1 to 12. 
 
The file `products.csv` comprises the list of products sold by the online store. It contains **46.294 different products** associated with **123 different subcategories**. Each record/line in the file has the following fields:

* **Item ID** - the unique identifier of the item. Could be represented as an integer number. 
* **Product Categories** - the category and subcategories of the item. It is a string containing the category and subcategories of the item. Eg. `appliances.kitchen.juice`


In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) and **[MLxtend](http://rasbt.github.io/mlxtend/)**. When using MLxtend, frequent patterns can either be discovered using `Apriori` and `FP-Growth`. **Choose the pattern mining algorithm to be used.** 


## Team Identification

**GROUP PD03**

Students:

* Eduardo Carvalho - nº55881
* Filipe Santos - nº55142
* Ivo Oliveira - nº50301

## 1. Mining Frequent Itemsets and Association Rules


In this first part of the project you should load and preprocess the dataset  in order to compute frequent itemsets and generate association rules considering all the sessions.

**In what follows keep the following question in mind and be creative!**

1. What are the most interesting products?
2. What are the most bought products?
3. Which products are bought together?
4. Can you find associations between the clicked products? 
5. Can you find associations highliting that when people buy a product/set of products also buy other product(s)?
6. Can you find associations highliting that when people click in a product/set of products also buy this product(s)?
7. Can you find relevant associated categories? 

### 1.1. Load and Preprocess Data

 **Product quantities should not be considered.**

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import  TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
#!pip install mlxtend
#from mlxtend.frequent_patterns import Fpgrowth

ImportError: ignored

In [21]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: ignored

In [23]:
products=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/products.csv',
                     header=None,
                     names=['ItemID','Category'],
                     dtype={'ItemID': int, 'Category':str})
buys=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/store-buys.dat', 
                 header=None,
                 names=['SessionID','TimeStamp','ItemID','Price','Qty'],
                 dtype={'SessionID':int, 'TimeStamp':str, 'ItemID': int, 
                        'Price':int, 'Qty':int})
clicks=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/store-clicks.dat', 
                   header=None,
                   names=['SessionID','TimeStamp','ItemID','Context'],
                   dtype={'SessionID':int, 'TimeStamp':str, 'ItemID': int, 
                        'Context':str})


,SessionID,TimeStamp,ItemID,Price,Qty
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1
...,...,...,...,...,...
201703,1869646,2014-04-29T04:30:17.701Z,214691390,1883,1
202064,2045326,2014-04-29T05:12:50.924Z,214826705,2303,1
202680,1939698,2014-04-29T04:11:23.373Z,214826610,732,1
202790,2078987,2014-04-29T04:37:32.711Z,214712235,11414,1


In [46]:
"""
DATABASE CORRECTIONS
products_no_duplicates - removed duplicates
buys_upd - added product names & season

rows with Price or Qty == 0 have been dropped
"""

products_no_duplicates = products.drop_duplicates().reset_index(drop=True)
buys_upd =pd.merge(buys,products_no_duplicates)

buys_upd = buys_upd[buys_upd.Price>0]
buys_upd = buys_upd[buys_upd.Qty>0]

product_name=[]
for cat in buys_upd.Category:
  product_name.append(cat.split('.')[-1].replace('_',' ').title())

product_name[:5]
buys_upd['ProductName']=product_name
buys_upd=buys_upd.drop('Category', axis=1)

dates =[]
season =[]

for i in buys_upd["TimeStamp"]:
  dates.append(i[:10])
  if i[5:7] == "04" or i[5:7] == "05":
     season.append("Spring")
  elif i[5:7] == "06" or i[5:7] == "07":
    season.append("Summer")
  else:
    season.append("Other")

weekday=[]
buys_upd["TimeStamp"] = pd.to_datetime(buys_upd["TimeStamp"])
buys_upd["Weekday_Num"]=buys_upd["TimeStamp"].dt.dayofweek 

for i in buys_upd["Weekday_Num"]:
  if i < 5: 
    weekday.append("Weekday")
  else :
    weekday.append("Weekend")

  
buys_upd = buys_upd.drop(columns=['TimeStamp'])
buys_upd.insert(1, "TimeStamp", dates)
buys_upd["Season"] = season
buys_upd["Weekday"] = weekday
buys_upd.sort_values(by='TimeStamp')
buys_upd.tail(10)


#unsure about what this does

#products=products.sort_values(by='ItemID').reset_index(drop=True)
#products.head()

,SessionID,TimeStamp,ItemID,Price,Qty,ProductName,Weekday_Num,Season,Weekday
390180,1550379,2014-04-26,214844728,2083,1,Vacuum,5,Spring,Weekend
390181,1550379,2014-04-26,214744757,941,1,Subwoofer,5,Spring,Weekend
390182,1350991,2014-04-28,214629122,17698,1,Joystick,0,Spring,Weekday
390185,1489537,2014-04-23,214691160,1360,1,Camera,2,Spring,Weekday
390190,1661893,2014-04-28,214664555,8900,1,Watering,0,Spring,Weekday
390191,1732031,2014-04-23,214586630,9947,1,Pillow,2,Spring,Weekday
390193,1412591,2014-04-26,214839705,11414,1,Tonometer,5,Spring,Weekend
390211,1551168,2014-04-23,214709289,1883,1,Ebooks,2,Spring,Weekday
427777,2086969,2014-04-29,214644716,1674,1,Water Heater,1,Spring,Weekday
427778,2086969,2014-04-29,214644716,1674,1,Water Heater,1,Spring,Weekday


In [ ]:
buys_upd=buys_upd.sort_values(by='SessionID').reset_index(drop=True)
buys_upd.head()

,SessionID,TimeStamp,ItemID,Price,Qty,ProductName,Season
0,11,2014-04-03,214821371,1046,1,Blender,Spring
1,11,2014-04-03,214821371,1046,1,Blender,Spring
2,12,2014-04-02,214717867,1778,4,Bag,Spring
3,21,2014-04-07,214548744,3141,1,Skates,Spring
4,21,2014-04-07,214838503,18745,1,Clocks,Spring


Existem 113 registos duplicados. Precisamos de os remover para avançar com a análise.

In [47]:
buys_upd=buys_upd.drop_duplicates().reset_index(drop=True)
len(buys_upd)

157184

In [48]:
clicks=clicks.sort_values(by='SessionID').reset_index(drop=True)
clicks.head()

,SessionID,TimeStamp,ItemID,Context
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0


In [49]:
len(clicks)-len(clicks.drop_duplicates())

23

Existem 46 registos duplicados. Precisamos de os remover para avançar com a análise.

In [50]:
clicks=clicks.drop_duplicates().reset_index(drop=True)

In [ ]:
#1. What are the most interesting products?
click_most_interesting = clicks.groupby("ItemID", as_index=False).size().sort_values(by="size", ascending=False)
click_most_interesting = click_most_interesting.head(10)

list_click_most_interesting = list(click_most_interesting["ItemID"])
most_int = pd.merge(click_most_interesting, products_no_duplicates)
most_int.head(10)

[643078800, 214829878, 214826610, 214834880, 214839973, 214748336, 214834877, 214835017, 214836932, 214821309]


,ItemID,size,Category
0,643078800,147419,computers.gaming
1,214829878,102563,sport.tennis
2,214826610,67473,computers.components.memory
3,214834880,61253,appliances.environment.fan
4,214839973,59083,medicine.tools.tonometer
5,214748336,55844,computers.components.videocards
6,214834877,53308,appliances.environment.fan
7,214835017,51347,appliances.kitchen.toster
8,214836932,48943,computers.peripherals.monitor
9,214821309,48791,appliances.kitchen.blender


The most interesting products are blabla



In [ ]:
#2. What are the most bought products?

df4 = pd.merge(buys, products_no_duplicates)

quantities = df4.groupby(["Category"], as_index=False).sum(["Qty"]).sort_values(by="Qty", ascending=False)

quantities = quantities[["Category","Qty"]]
quantities = quantities.rename(columns={"Qty": "Quantity"})
quantities = quantities.reset_index(drop=True)

quantities.head(10)


,Category,Quantity
0,appliances.kitchen.blender,40163
1,computers.components.memory,23810
2,sport.tennis,16864
3,appliances.kitchen.meat_grinder,11100
4,electronics.video.tv,9055
5,appliances.environment.fan,7387
6,medicine.tools.tonometer,6156
7,computers.notebook,5667
8,appliances.iron,5198
9,accessories.bag,5066


The most bought product category are Blenders

## 1.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

In [ ]:
#3. Which products are bought together?
all_sessions={}
for i in range(len(buys_upd)):
  all_sessions[buys_upd.SessionID[i]]=[]

for i in range(len(buys_upd)):
  if buys_upd.ProductName[i] not in all_sessions.get(buys_upd.SessionID[i]):
    all_sessions[buys_upd.SessionID[i]].append(buys_upd.ProductName[i])

transactions=list(all_sessions.values())

In [ ]:
transactions[:5]

[['Blender'],
 ['Blender', 'Meat Grinder'],
 ['Blender', 'Memory'],
 ['Blender'],
 ['Blender', 'Memory']]

In [ ]:
#Compute binary database
tr_enc = TransactionEncoder()
trans_array = tr_enc.fit(transactions).transform(transactions)
binary_database = pd.DataFrame(trans_array, columns=tr_enc.columns_)
binary_database.head(3)

,Acoustic,Air Conditioner,Air Heater,Alarm,Anti Freeze,Bag,Bath,Battery,Bed,Bicycle,Blanket,Blender,Bottles,Cabinet,Calculator,Camera,Carriage,Cartrige,Cdrw,Chair,Climate,Clocks,Coffee Grinder,Coffee Machine,Compressor,Cooler,Costume,Cpu,Cultivator,Desktop,Diapers,Dictaphone,Dishwasher,Diving,Dolls,Drill,Ebooks,Fan,Faucet,Fryer,...,Scanner,Screw,Sewing Machine,Shelving,Shirt,Shoes,Skates,Ski,Smartphone,Snowboard,Sock,Sofa,Sound Card,Stapler,Steam Cleaner,Steam Cooker,Subwoofer,Swing,Table,Tablet,Telephone,Tennis,Toilet,Tonometer,Toster,Toys,Trainer,Trousers,Tshirt,Tv,Umbrella,Vacuum,Video,Videocards,Videoregister,Washer,Water Heater,Watering,Weather Station,Welding
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#Compute itemsets min_support = 1%
frequent_itemsets = apriori(binary_database, min_support=0.001, use_colnames=True)

# add new column length
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# filter using pattern length = 2
frequent_2_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].reset_index(drop=True)
frequent_2_itemsets

,support,itemsets,length
0,0.001130,"(Blender, Bag)",2
1,0.001925,"(Bag, Memory)",2
2,0.001608,"(Tennis, Bag)",2
3,0.001454,"(Tv, Bag)",2
4,0.002000,"(Camera, Blanket)",2
...,...,...,...
203,0.001652,"(Lawn Mower, Tennis, Toster)",3
204,0.001743,"(Tennis, Monitor, Memory)",3
205,0.002223,"(Toster, Monitor, Memory)",3
206,0.002807,"(Tennis, Toster, Memory)",3


In [ ]:
#FP-Growth é melhori que apriori

#fpgrowth(binary_database, min_support=0.6,use_colnames=True)

### 1.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 1.4. Take a Look at Maximal Patterns: Compute Maximal Frequent Itemsets

### 1.5. Conclusions 

# 2. Week vs Weekend Purchases

In this part of the project you should analyse the consumption patterns during the week vs during the weekeed.

**In what follows keep the following question in mind and be creative!**

1. The most interesting products are the same during the week and the weekend? 
2. What are the most bought products during the week? And during the weekend?
3. There are differences between the sets of products bought during the week and the weekend?
4. Can you find different associations highliting that when people click in a product/set of products also buy this product(s) during the week vs the weekend?
5. Discuss the results obtained for the week sessions vs weekend sessions.

### 2.1. Load and Preprocess Data

 **Product quantities should not be considered.**

### 2.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

### 2.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 2.4. Conclusions 

# 3. [Only Groups of 3] Spring vs Summer Purchases

In this part of the project you should analyse the consumption patterns during the Spring months (April and May) vs Summer months (June and July).

**In what follows keep the following question in mind and be creative!**

1. The most interesting products are the same during the Spring and the Summer? 
2. What are the most bought products during the Spring? And during the Summer?
3. There are differences between the sets of products bought during the Spring and the Summer?
4. Can you find different associations highliting that when people click in a product/set of products also buy this product(s) during the Spring vs the Summer?
5. Discuss the results obtained for the Spring sessions vs Summer sessions.

### 3.1. Load and Preprocess Data

 **Product quantities should not be considered.**

### 3.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

### 3.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 3.4. Conclusions 

## 4. Conclusions
Draw some conclusions about this project work.